In [1]:
import os
import vedo
import numpy as np
import trimesh
from copy import copy
from scipy.special import sph_harm
import ipywidgets as widgets
from ipywidgets import interact
import pyvista as pv
from IPython import embed
import re

# Spherical harmonics

The spherical harmonics are defined by the following formula

$$Y_{lm}(\theta, \phi)=\sqrt{\frac{2l+1}{4\pi}}P_l^m(\cos\theta)e^{im\phi},$$

where the $P_l^m$ are the associated Legendre polynomials. They satisfy the following equation:

$$r^2\nabla^2Y_{lm}(\theta, \phi)=l(l+1)Y_{lm}(\theta, \phi)$$

In other words, the Laplacian operator $\nabla^2$ can be diagonalized by using this basis. The associated matrix for the first 1+3+5 spherical harmonics is:

$$\begin{pmatrix}
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 2 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 2 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 6 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 6 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 6 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 6 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 6\\
\end{pmatrix}$$

The deformation is performed along the radial direction at each point, and with an amplitude given by the following expansion:

$$ \textbf{d}(\theta,\phi,t) =\bigg( \sum_{l=0}^{l_{\text{max}}}\sum_{m=-l}^{l}a_{lm}(t)Y_{lm}(\theta, \phi)\bigg) \hat{\textbf{r}}$$

$$a_{lm}(t)=b_{lm}^{(0)}+\sum_{n=1}^{N_t}a_{lm}^{(n)}\sin(2\pi nt)+b_{lm}^{(n)}\cos(2\pi nt),\,\, t\in[0,1]$$

### A local deformation

The expansion of the Dirac delta is given by:

$$\delta(\theta,\phi)=\sum_{l=0}^{\infty}\sum_{m=-l}^{l}Y_{lm}(0, 0)Y_{lm}(\theta, \phi)$$

Would this be useful to create a local deformation (by truncating this expansion)?

### How to perform content and style deformations

One possibility is to generate a basis of spherical harmonics that spans the subspace of possible content deformations; and another that spans the subspace of style deformations.

$$ \textbf{d}_c(\theta,\phi)\in span(\mathcal{B}_c)$$
$$ \textbf{d}_s(\theta,\phi)\in span(\mathcal{B}_s)\subseteq span(\mathcal{B}_c)^{\bot}$$

The style deformation is such that it varies with time in a periodical manner. Therefore, the overall deformation will be of the form

$$ \textbf{d}(\theta,\phi,t) = \textbf{d}_c(\theta,\phi) + A(t)\textbf{d}_s(\theta,\phi) $$ with $$A(t)=A(t+2\pi)$$


For example, we can make $$A(t)=A_0\sin t$$

The generative process is such that:

$$a_{lm}^{(n)}\sim\mathcal{N}(0,1)$$

for each $l$, $m$ and $n$. These $a_{lm}^{(n)}$ are the latent variables of the model.

### Election of content and style subspaces

Let's call $E_l$ the eigenspaces of the laplacian with eigenvalues $l(l+1)$ and let's make, arbitrarily

$$\mathcal{S}_c=E_1\oplus E_2$$
$$\mathcal{S}_s=E_3\oplus E_4$$

where $\oplus$ represents the direct sum of vector spaces.

_____________

## `DeformedSphere` class

This is a first attempt to create a class that models the deformation applied to a sphere, in the form of the spherical harmonics from above

In [2]:
def appendSpherical_np(xyz):
      
      '''
      params:
          xyz: NumPy array representing the (x,y,z) coordinates for a point cloud.
          
      return:
          A NumPy array with 6 columns containing the input (x,y,z) coordinates
          and, additionally, the (r, theta, phi) spherical coordinates        
      '''
      
      ptsnew = np.hstack((xyz, np.zeros(xyz.shape)))
      xy = xyz[:,0]**2 + xyz[:,1]**2        
      ptsnew[:,3] = np.sqrt(xy + xyz[:,2]**2)
      ptsnew[:,4] = np.arctan2(np.sqrt(xy), xyz[:,2]) # for elevation angle defined from Z-axis down      
      ptsnew[:,5] = np.arctan2(xyz[:,1], xyz[:,0])
      return ptsnew

In [3]:
class DeformedSphere(trimesh.Trimesh):
    
    def __init__(self, reference_shape=vedo.Sphere().to_trimesh()):
        
        super().__init__(reference_shape.vertices, reference_shape.faces)        
        self.sphere_coords = self._appendSpherical(self.vertices)[:,3:]
                
    def deform(self, amplitude, l, m):
        
        '''
        This method returns a deformed sphere, using the spherical harmonic Ylm with an amplitude
        
        params:
            l, m: indices of the spherical harmonic (Ylm).
            amplitude: amplitude of the spherical harmonic.
        '''
        
        coefs = np.array([
            1 + amplitude * sph_harm(m, l, *self.sphere_coords[k, 1:3]).real
            for k in range(self.sphere_coords.shape[0])
        ])
        
        self.vertices = np.multiply(
            self.vertices, 
            np.kron(np.ones((3,1)), coefs).transpose()
        )
        
        return self
        
    # https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion
    def _appendSpherical(self, xyz):        
        return appendSpherical_np(self.xyz)       

In [4]:
polos = [(x,y,z) for x in [-1,0,1] for y in [-1,0,1] for z in [-1,0,1] if x**2+y**2+z**2==1]
appendSpherical_np(np.array(polos))[:,-2:]

array([[ 1.57079633,  3.14159265],
       [ 1.57079633, -1.57079633],
       [ 3.14159265,  0.        ],
       [ 0.        ,  0.        ],
       [ 1.57079633,  1.57079633],
       [ 1.57079633,  0.        ]])

# <span style="color:red">Development</span>

In [5]:
T = 20
freq_max = 2
l_max = 2

$$a_{lm}(t)=b_{lm}^{(0)}+\sum_{n=1}^{N}a_{lm}^{(n)}\sin(2\pi nt)+b_{lm}^{(n)}\cos(2\pi nt)$$

We cache 
$$f_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\sin(2\pi nt)$$
$$g_{lmn}(\theta,\phi,t)=Y_{lm}(\theta,\phi)\cos(2\pi nt)$$

We build a dictionary  $Y_{lm}(\theta, \phi)$ for all the $(\theta, \phi)$ in the discretization of $S_2$. First we get the coordinates of such discretization:

In [6]:
sphere = vedo.Sphere().to_trimesh()
sphere_coords = appendSpherical_np(sphere.vertices)[:,3:]

In [7]:
def cache_sin_and_cos(freq_max, Nt):
    '''
    Generate spherical harmonics for a set of locations across the sphere.
    params:
       freq_max: maximum multiple of the fundamental frequency.
       Nt: number of equispaced time points on the [0,2*pi] interval
    returns:
       two dictionaries in a tuple, where the keys of each are the n indices 
       of the sine and cosine functions
    '''
    
    sin_n = { 
        w: np.array([np.sin(2*w*np.pi*i/Nt) for i in range(Nt)]) 
        for w in range(1,freq_max) 
    }
    
    cos_n = { 
        w: np.array([np.cos(2*w*np.pi*i/Nt) for i in range(Nt)])
        for w in range(1,freq_max) 
    }

    return sin_n, cos_n

In [8]:
def cache_Ylm(sphere_coords, l_max):
    '''
    Generate spherical harmonics for a set of locations across the sphere.
    params:
       sphere_coords: numpy.array of shape (M, 2) where M is the number of points
       indices: list of (l,m) indices
    returns:
       a dictionary where the keys are the (l,m) indices 
       and the values are real spherical harmonics at the
       (theta, phi) angles in sphere_coords
    '''
    
    Y_lm = {}
    
    indices = [ (l,m) for l in range(l_max+1) for m in range(-l,l+1) ]
    for l, m in indices:
        
        # Real spherical harmonics (defined in terms of the complex-valued ones)
        if m < 0:
            Y_lm[(l,m)] = np.array([
              np.sqrt(2) * sph_harm(-m, l, *sphere_coords[k, 1:3]).imag
              for k in range(sphere_coords.shape[0])      
            ])
        elif m > 0:
            Y_lm[(l,m)] = np.array([
              np.sqrt(2) * sph_harm(m, l, *sphere_coords[k, 1:3]).real
              for k in range(sphere_coords.shape[0])
            ])
        else:
            Y_lm[(l,m)] = np.array([
              sph_harm(m, l, *sphere_coords[k, 1:3]).real
              for k in range(sphere_coords.shape[0])
            ])
            
    return Y_lm

In [18]:
def cache_base_functions(sphere_coords, l_max, freq_max, Nt):
        
    sin_n, cos_n = cache_sin_and_cos(freq_max, Nt)
    Y_lm = cache_Ylm(sphere_coords, l_max)
    
    f_lmn = {}
    g_lmn = {}
    
    for l, m in Y_lm:
        f_lmn[l,m] = {}
        g_lmn[l,m] = {}
        for n in sin_n:
             f_lmn[l,m][n] = np.dot( np.expand_dims(Y_lm[l,m],1), np.expand_dims(sin_n[n], 0) )       
             g_lmn[l,m][n] = np.dot( np.expand_dims(Y_lm[l,m],1), np.expand_dims(cos_n[n], 0) )
    
    return Y_lm, f_lmn, g_lmn

Sample coefficients $a_{lm}^{(n)}\sim \mathcal{N}(0,\sigma^2)$.

In [19]:
import random

def sample_coefficients(l_max, freq_max=None, amplitude_max=0.1, random_seed=None):
        
    if random_seed is not None:
       random.seed(random_seed)
        
    if freq_max is not None:
      amplitude_lmn = {"sin":{}, "cos":{}}
      for n in range(1,freq_max):
        for l in range(l_max+1):
           for m in range(-l,l+1):
             amplitude_lmn["cos"][(l,m,n)] = random.gauss(0, amplitude_max)     
             amplitude_lmn["sin"][(l,m,n)] = random.gauss(0, amplitude_max)     
      return amplitude_lmn

    else:
      amplitude_lm = {}      
      for l in range(l_max+1):
         for m in range(-l, l+1):
           amplitude_lm[(l,m)] = random.gauss(0, amplitude_max)     
      return amplitude_lm    

In [20]:
Y_lm, f_lmn, g_lmn = cache_base_functions(sphere_coords, l_max=2, freq_max=2, Nt=T)

In [21]:
def generate_population(N, T, amplitude_static_max, amplitude_dynamic_max, random_seed, verbose=False):

    mesh_list = []
    coefs = []    
    
    # Loop through individuals
    for i in range(N):

        if verbose:
            print(i)

        # Generate latent variables for a single individual
        mesh_i = []
        amplitude_lm = sample_coefficients(l_max, amplitude_max=amplitude_static_max)
        amplitude_lmn = sample_coefficients(
            l_max, freq_max, amplitude_max=amplitude_dynamic_max
        )
        coefs.append([amplitude_lm, amplitude_lmn])
        #####

        deformations = np.ones(sphere_coords.shape[0])
        
        # Perform static (content) deformations
        for l, m in Y_lm:
            deformations += amplitude_lm[l, m] * Y_lm[l, m]

        # Loop through time to perform dynamic (style) deformations
        for t in range(T):
            for l, m, n in amplitude_lmn["sin"]:
                deformations += amplitude_lmn["sin"][l, m, n] * f_lmn[l, m][n][:, t]
                deformations += amplitude_lmn["cos"][l, m, n] * g_lmn[l, m][n][:, t]

            deformed_sphere = vedo.Sphere(res=24).to_trimesh()

            deformed_sphere.vertices = np.multiply(
                deformed_sphere.vertices,
                np.kron(np.ones((3, 1)), deformations).transpose(),
            )

            mesh_i.append(deformed_sphere)

        mesh_list.append(mesh_i)

    return mesh_list, coefs



### Generate population

In [22]:
AMPLITUDE_STATIC_MAX = 0.2
AMPLITUDE_DYN_MAX = 0.02
N = 1000

population, coefs = generate_population(
    N, T, 
    amplitude_static_max = AMPLITUDE_STATIC_MAX, 
    amplitude_dynamic_max = AMPLITUDE_DYN_MAX, 
    random_seed=1, verbose=True
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [23]:
kk = np.array(
    [
        np.array([np.array(tf.vertices) for tf in individual])
        for individual in population
    ]
)

import pickle as pkl

with open("synthetic_population.pkl", "wb") as ff:
    pkl.dump({"population_meshes": kk, "coefficients": coefs}, ff)


### Mesh visualization with Pyvista

In [ ]:
conn = vedo.Sphere().to_trimesh().faces
conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3

def generate_gif(mesh4D, filename, camera_position='xy'):
    
    pv.set_plot_theme("document")
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    
    # plotter = pv.Plotter(shape=(1, len(camera_positions)), notebook=False, off_screen=True)
    plotter = pv.Plotter(notebook=False, off_screen=True)
        
    # Open a gif
    plotter.open_gif(filename) 

    kk = pv.PolyData(np.array(mesh4D[0].vertices), conn)
    # plotter.add_mesh(kk, smooth_shading=True, opacity=0.5 )#, show_edges=True)
    plotter.add_mesh(kk, show_edges=True) 
    
    for t, _ in enumerate(mesh4D):
        kk = pv.PolyData(np.array(mesh4D[t].vertices), conn)
        plotter.camera_position = camera_position
        plotter.update_coordinates(kk.points, render=False)
        plotter.render()             
        plotter.write_frame()
    
    plotter.close()
    
    
def generate_gif_population(population, N_gifs=10, camera_positions=['xy', 'yz', 'xz']):
    for i, moving_mesh in enumerate(population):
        print(i)
        for camera_position in camera_positions:        
            filename = "gifs/subject{}_{}.gif".format(i, camera_position)
            generate_gif(moving_mesh, filename, camera_position)        
        if i == N_gifs:
            break

In [ ]:
generate_gif_population(population)

____________

In [ ]:
def apply_deformation(sphere, spherical_coords, amplitude_lm):
    
    deformed_sphere = sphere
    
    coefs = np.ones(spherical_coords.shape[0])
    
    for l, m in amplitude_lm:        
        coefs += amplitude_lm[l, m] * Y_lm[l,m]
                
    deformed_sphere.vertices = np.multiply(
        deformed_sphere.vertices, np.kron(np.ones((3, 1)), coefs).transpose()
    )
    
    return deformed_sphere


In [ ]:
def get_coef_as_string(l,m):
    return "a" + str(l) + str(m).replace("-","_")

def get_lm(string):
    import re    
    regex = re.compile(".*([0-9])(_?[0-9])")
    l = int(regex.match(string)[1])
    m = int(regex.match(string)[2].replace("_","-"))
    return l,m    

In [ ]:
conn = vedo.Sphere().to_trimesh().faces
conn = np.c_[np.ones(conn.shape[0]) * 3, conn].astype(int)  # add column of 3

def f(**kwargs):

    coefs = {get_lm(lm):kwargs[lm] for lm in kwargs}
    
    sphere = vedo.Sphere().to_trimesh()
    spherical_coords = _appendSpherical_np(sphere.vertices)[:,3:]
    deformed_sphere = apply_deformation(sphere, spherical_coords, coefs)
        
    mesh = pv.PolyData(deformed_sphere.vertices, conn)
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=True)
    pl.add_mesh(mesh, show_edges=True)
    pl.show(use_panel=True, interactive=True, interactive_update=True)

In [ ]:
coefs = {get_coef_as_string(*lm): widgets.FloatSlider(min=-0.5, max=0.5) for lm in indices}
interact(f, **coefs);

In [ ]:
# t = 15
# mesh_list[t].show(faces=True)

In [ ]:
#def f(l, m, amplitude):
#    return DeformedSphere().deform(amplitude, l, m).show()

#### Saving the synthetic meshes as `obj` files

In [ ]:
# odir = "/home/rodrigo/tmp/mallas_obj"
# os.makedirs(odir, exist_ok=True)
# 
# N = 100
# 
# for i in range(N):
#     for t, mesh in enumerate(mesh_list):
#         
#         # print(mesh.vertices)
#         j = str(j)
#         j = "0" * (2 - len(j)) + j # 5 --> 05, 15 --> 15
#         
#         with open(os.path.join(odir, "sphere_{}.obj".format(i)), "wt") as ff:        
#             obj_content = trimesh.exchange.export.export_obj(
#                 mesh, 
#                 include_normals=False, 
#                 include_texture=False
#             )
#             ff.write(obj_content)
#             
#         # print(mesh.show())

__________________________

# Attempt to visualize meshes (to complete)

#### `itkwidgets`

In [ ]:
import itk
import itkwidgets
from itkwidgets import view

#### `Trimesh`

In [ ]:
import trimesh

#### `pyvista`

#### `polyscope`

In [ ]:
import polyscope as ps
ps.init()
mesh = mesh_list[25]
ps.register_surface_mesh("sph", mesh.vertices, mesh.faces, smooth_shade=True)
ps.show()

________________________

In [ ]:
DeformedSphere().deform(0, 0, 0).vertices

In [ ]:
interact(f, t=widgets.IntSlider(N_t-1, max=N_t-1));

In [ ]:
# np.polynomial.chebyshev.Chebyshev((1,2,3))()